<a href="https://colab.research.google.com/github/Jimmy586/DataAnalysis_ProbabilisticModeling/blob/main/Fuzzy_matching_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def levdist(s,t):
  if s == '':
    return len(t)
  if t == '':
    return len(s)
  if s[-1] == t[-1]:
    cost =0
  else:
    cost =1

  dist = min([levdist(s[:-1],t)+1, levdist(s, t[:-1])+1, levdist(s[:-1],t[:-1])+ cost])

  return dist

In [5]:
levdist('irks', 'risk')

3

In [2]:
import time

st = time.time()
s1= 'micrsft'
s2= 'microsoft corporation'
print('Levenshtein distance between : ', f'{s1}', ' and ', f'{s2}',' is ', levdist(s1,s2))
et = time.time()
print(et-st)

Levenshtein distance between :  micrsft  and  microsoft corporation  is  14
32.84070324897766


**This is so slow**

In [3]:
!pip install  jellyfish

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.1 MB/s eta 0:00:00


In [5]:
import jellyfish
st = time.time()
print('Levenshtein distance is :',jellyfish.levenshtein_distance(s1,s2))
et = time.time()
print(et-st)
st = time.time()
print('Damarau Levenshtein distance is :',jellyfish.damerau_levenshtein_distance(s1,s2))
et = time.time()
print(et-st)

Levenshtein distance is : 14
0.002551555633544922
Damarau Levenshtein distance is : 14
0.0001933574676513672


**N-GRAM Algorithm**

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = ['company name is microsoft', 'company the name is Microsoft',
          'company name mcrosft', 'the company is Microsft Co',
         'company is Microsoft Corporation', 'the company is name microsoft Corp',
          'company MCSFT CO name']

clean_corpus = ['The company name is Microsoft Corporation']

In [9]:
vect1 = TfidfVectorizer(analyzer='word', ngram_range=(1, 1))
x = vect1.fit_transform(corpus)

print(vect1.get_feature_names_out())

['co' 'company' 'corp' 'corporation' 'is' 'mcrosft' 'mcsft' 'microsft'
 'microsoft' 'name' 'the']


In [10]:
vect2 = TfidfVectorizer(analyzer='word', ngram_range=(3, 3))
x = vect2.fit_transform(corpus)

print(vect2.get_feature_names_out())

['company is microsft' 'company is microsoft' 'company is name'
 'company mcsft co' 'company name is' 'company name mcrosft'
 'company the name' 'is microsft co' 'is microsoft corporation'
 'is name microsoft' 'mcsft co name' 'name is microsoft'
 'name microsoft corp' 'the company is' 'the name is']


In [11]:
vect3 = TfidfVectorizer(analyzer='word', ngram_range=(2, 5))
x = vect3.fit_transform(corpus)

print(vect3.get_feature_names_out())

['co name' 'company is' 'company is microsft' 'company is microsft co'
 'company is microsoft' 'company is microsoft corporation'
 'company is name' 'company is name microsoft'
 'company is name microsoft corp' 'company mcsft' 'company mcsft co'
 'company mcsft co name' 'company name' 'company name is'
 'company name is microsoft' 'company name mcrosft' 'company the'
 'company the name' 'company the name is' 'company the name is microsoft'
 'is microsft' 'is microsft co' 'is microsoft' 'is microsoft corporation'
 'is name' 'is name microsoft' 'is name microsoft corp' 'mcsft co'
 'mcsft co name' 'microsft co' 'microsoft corp' 'microsoft corporation'
 'name is' 'name is microsoft' 'name mcrosft' 'name microsoft'
 'name microsoft corp' 'the company' 'the company is'
 'the company is microsft' 'the company is microsft co'
 'the company is name' 'the company is name microsoft' 'the name'
 'the name is' 'the name is microsoft']


**Let's implement on this example and compare with Levenshtein distance**

In [29]:
!pip install fuzzy_match

In [14]:
import pandas as pd
import itertools

# for fuzzy matching
from fuzzy_match import algorithims

In [15]:
# make a cartesian combination of the words in the corpus and the correct word

cart_list = list(itertools.product(corpus, clean_corpus))
cart_list

[('company name is microsoft', 'The company name is Microsoft Corporation'),
 ('company the name is Microsoft',
  'The company name is Microsoft Corporation'),
 ('company name mcrosft', 'The company name is Microsoft Corporation'),
 ('the company is Microsft Co', 'The company name is Microsoft Corporation'),
 ('company is Microsoft Corporation',
  'The company name is Microsoft Corporation'),
 ('the company is name microsoft Corp',
  'The company name is Microsoft Corporation'),
 ('company MCSFT CO name', 'The company name is Microsoft Corporation')]

In [16]:
def get_matches(lst):

    '''function to calculate the Levenshtein
    distance, trigram match score and the cosine
    similarity matches for the above example'''

    x, y, lscore, tscore, cscore = [], [], [], [], []

    for i in range(len(lst)):
        x.append(lst[i][0])
        y.append(lst[i][1])

        lscore.append(round(algorithims.levenshtein(lst[i][0], lst[i][1]), 3))
        tscore.append(round(algorithims.trigram(lst[i][0], lst[i][1]), 3))
        cscore.append(round(algorithims.cosine(lst[i][0], lst[i][1]), 3))

    df = pd.DataFrame(list(zip(x, y, lscore, tscore, cscore)),
                    columns = ['in_data', 'clean_data', 'lev_score', 'tri_score', 'cosine_score'])

    return df

In [30]:
!pip install fuzzywuzzy
!pip install opendatasets

In [17]:
get_matches(cart_list)

,in_data,clean_data,lev_score,tri_score,cosine_score
0,company name is microsoft,The company name is Microsoft Corporation,0.585,0.650,0.612
1,company the name is Microsoft,The company name is Microsoft Corporation,0.512,0.750,0.730
2,company name mcrosft,The company name is Microsoft Corporation,0.463,0.386,0.471
3,the company is Microsft Co,The company name is Microsoft Corporation,0.610,0.512,0.365
4,company is Microsoft Corporation,The company name is Microsoft Corporation,0.780,0.775,0.816
5,the company is name microsoft Corp,The company name is Microsoft Corporation,0.634,0.780,0.500
6,company MCSFT CO name,The company name is Microsoft Corporation,0.293,0.333,0.408


# REAL WORLD DATA

In [22]:
import opendatasets as od
link ='https://www.kaggle.com/datasets/leandrodoze/room-type'
od.download(link)

100%|██████████| 1.53k/1.53k [00:00<00:00, 2.04MB/s]

In [26]:
import pandas as pd
data_df =pd.read_csv('/content/room-type/room_type.csv')

In [27]:
data_df.head()

,Expedia,Booking.com
0,"Deluxe Room, 1 King Bed",Deluxe King Room
1,"Standard Room, 1 King Bed, Accessible",Standard King Roll-in Shower Accessible
2,"Grand Corner King Room, 1 King Bed",Grand Corner King Room
3,"Suite, 1 King Bed (Parlor)",King Parlor Suite
4,"High-Floor Premium Room, 1 King Bed",High-Floor Premium King Room


In [32]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 12.2 MB/s eta 0:00:00


In [41]:
from fuzzywuzzy import fuzz
from fuzzy_match import algorithims
def get_ratio(row):
  col1 = row['Expedia']
  col2= row['Booking.com']
  return fuzz.partial_ratio(col1,col2)

In [34]:
def get_partial_ratio(row):

    col1 = row['Expedia']
    col2 = row['Booking.com']

    return fuzz.partial_ratio(col1, col2)

In [35]:
def get_token_sort_ratio(row):
    col1 = row['Expedia']
    col2 = row['Booking.com']

    return fuzz.token_sort_ratio(col1, col2)

In [37]:
def get_token_set_ratio(row):

    col1 = row['Expedia']
    col2 = row['Booking.com']

    return fuzz.token_set_ratio(col1, col2)

In [39]:
def get_trigram_value(row):

    col1 = row['Expedia']
    col2 = row['Booking.com']

    return round(algorithims.trigram(col1, col2), 3)

In [43]:
data_df['full_ratio'] = data_df.apply(get_ratio, axis=1)
data_df['partial_ratio'] = data_df.apply(get_partial_ratio, axis=1)
data_df['token_sort_ratio'] = data_df.apply(get_token_sort_ratio, axis=1)
data_df['token_set_ratio'] = data_df.apply(get_token_set_ratio, axis=1)
data_df['trigram'] = data_df.apply(get_trigram_value, axis=1)
data_df

,Expedia,Booking.com,full_ratio,partial_ratio,token_sort_ratio,token_set_ratio,trigram
0,"Deluxe Room, 1 King Bed",Deluxe King Room,69,69,84,100,0.739
1,"Standard Room, 1 King Bed, Accessible",Standard King Roll-in Shower Accessible,65,65,78,81,0.562
2,"Grand Corner King Room, 1 King Bed",Grand Corner King Room,100,100,80,100,0.793
3,"Suite, 1 King Bed (Parlor)",King Parlor Suite,65,65,85,100,0.750
4,"High-Floor Premium Room, 1 King Bed",High-Floor Premium King Room,82,82,90,100,0.829
...,...,...,...,...,...,...,...
98,"Room, 1 King Bed, Accessible, Resort View (Ali...",Alii Tower Resort View With King Bed - Mobilit...,37,37,68,83,0.543
99,"Room, 1 King Bed, Accessible, View (Rainbow, B...",Rainbow Tower Ocean View With King Bed - Mobil...,49,49,62,81,0.479
100,"Room, 1 King Bed, Ocean View (Alii)",Alii Tower Ocean View With King Bed,49,49,79,87,0.581
101,"Room, 1 King Bed, Oceanfront (Rainbow)",Rainbow Tower Ocean Front with King Bed,37,37,66,68,0.542
